# Area Processing (Tutorial 2)
***
# Table of Contents
1.   [Imports](#Imports)
2.   [Image Analysis](#Images-Analysis)
3.   [Exercise 1 - Sliding Window](#Exercise-1---Sliding-Window)
4.   [Exercise 2 - Convolution on RoI](#Exercise-2---Convolution-on-RoI)
5.   [Exercise 3 - Convolution on the Whole Image](#Exercise-3---Convolution-on-the-Whole-Image)
6.   [Exercise 4 - Different Convolution Kernels](#Exercise-4---Different-Convolution-Kernels)


# Imports

Only 4 libraries are needed for this project:
* opencv (cv2) - For image processing
* numpy - For its arrays
* matplotlib - Plotting histograms
* os - File traversal
* tqdm.notebook - tqdm progress bars, but for ipynb files
* Classes - Custom classes written by me for this assignment

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm
from Classes import Window, Sobel, Gaussian, Bilinear

# Image Analysis

### aiden dp.jpg

This is a picture of me in the upper barakka gardens in Valletta. The focus is myself with the background of the Grand
Harbour. The picture is lighted by natural lighting and includes a lot of non linear features.

<img src="images/aiden dp.jpg">

### cursed.jpg

The image I will be using for this lab is one of my failed attempts of creating a pie chart for one of my statistics units
from last year. It features a sun like object in the centre with black lines coming out from the centre, it is coloured in
full RGB while the background is gray.

<img src="images/cursed.jpg">

### dog chaos.png

This is a blurry picture of two of my friends and another friend's dog. They are in a dimly lit, tiled floor room with a
white wall and some furniture behind one of my friends.

<img src="images/dog chaos.png">

### jake car.png

This is a picture of my classmate Jake standing next to a red car. The photo was taken in a street during daytime. The
background features several home facades, 2 other cars and a person.

<img src="images/jake car.png">

### jake close up.jpg

This is a picture of my classmate Jake with a beige background.

<img src="images/jake close up.jpg">

### jake sitting.jpg

This is a picture of my classmate Jake sitting on the floor of a bathroom. The room is well lit and tiled all over.

<img src="images/jake sitting.jpg">

### jojo ben.jpg

This is a picture of my classmate Ben walking towards the Hal Ghaxaq church with a bag of fried chicken in his hand. The
photo was taken during the night so the lighting comes from old street lamps.

<img src="images/jojo ben.jpg">

#### I have permission by all the people shown to use these images for this tutorial

Here I load the images into a list

In [2]:
raw_images = []
for file in tqdm(os.listdir("images"), desc='Loading Images'):
    raw_images.append(cv2.imread("images/" + file))

Loading Images:   0%|          | 0/7 [00:00<?, ?it/s]

# Exercise 1 - Sliding Window

### Window Class

For this exercise I wrote the window class where I define some properties for the window.

```python
def __init__(self, image, n, s):
    self.x_boundary = image.shape[1] + n
    self.y_boundary = image.shape[0] + n
    self.top_left = (0, 0)
    self.bot_right = (n, n)
    self.previousBotY = n
    self.height = n
    self.stride = (s, s)
    try:
        self.channels = image.shape[2]
    except:
        self.channels = 1
```

To create a Window, the image (min 2d numpy array), n (length or width) and s(stride or step) are required.

Using these parameters I define:
* The x and y boundaries for the window.
* The starting top left and bottom right location as a tuple of two positions, where [0] is x and [1] is y.
* Previous y position, this is used to check if the window has changed it's y position.
* Height
* Stride
* Number of channels

Each window is a square. The object is intended to be used for the image passed in initialisation.

### Using Window for Ex 1

For this exercise I use 4 functions from the class:

```Python
def getPos(self):
    return self.top_left, self.bot_right
```

getPos returns the current position of the Window

```python
def forwardPos(self):
    # Case when you need to go down and start new line
    if (self.bot_right + self.stride)[0] >= (self.x_boundary - self.height):
        return (0, self.top_left[1] + self.stride[1]), (self.height, self.bot_right[1] + self.stride[1])
    # Generic move right case
    else:
        return (self.top_left[0] + self.stride[0], self.top_left[1]), \
               (self.bot_right[0] + self.stride[0], self.bot_right[1])
```

forwardPos returns the would be position of the next move. There are two cases:
1. Next step stays in X boundary and so the new positions are just changed by adding stride
2. Special case when next step would exceed X boundary so x positions are reset to 0, n and y positions are incremented
by stride

```python
def forwardMove(self):
    # Change positions
    self.top_left, self.bot_right = self.forwardPos()
    return self.top_left, self.bot_right
```

forwardMove changes the window's position to the return of forwardPos

```python
def inBoundary(self, new_top_left=None, new_bot_right=None):
    # Use current position if no new positions are passed
    if new_top_left is None:
        new_top_left = self.top_left
    if new_bot_right is None:
        new_bot_right = self.bot_right
    # Check if parameters are in boundary of the image given in initialisation
    return new_bot_right[0] <= self.x_boundary and new_bot_right[1] <= self.y_boundary and \
           new_top_left[0] >= 0 and new_top_left[1] >= 0
```

inBoundary returns whether given positions, or the current positions are inBoundary of the image.

### Code Explenation

First I initialise a Window for 'aiden dp.png' n=100, s=50. The reason for it being quite a "large" window is so that
the sliding window demonstration can go fast.

The starting and future positions are read using getPos and forwardPos. It is expected that win is initialised in boundary.

Then I loop while win is in its boundary.

* Every iteration I draw a rectangle on the image using cv2.rectangle.
* If show is on I display this using imshow.
* Then the positions are moved forwardMove and the future positions are taken again using forwardPos.

In [3]:
%%capture
# On/Off switch
show = True

win = Window(raw_images[0], 100, 50)
start_point, end_point = win.getPos()

new_tl, new_br = win.forwardPos()
while win.inBoundary(new_br):
    image = cv2.rectangle(raw_images[0].copy(), start_point, end_point, (255, 255, 255))
    if show:
        cv2.imshow("Sliding Window", image)
        cv2.waitKey(int(1/35*1000))
    start_point, end_point = win.forwardMove()
    new_tl, new_br = win.forwardPos()
cv2.destroyAllWindows()

# Exercise 2 - Convolution on RoI

In [4]:
# sobel = Sobel()
#
# win = Window(gray, 300, 1)
#
# image = cv2.imread("1mb pic.png")
# image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#
# roi = win.getImageInBoundary(image)
# cv2.imwrite("Output/roi_before_filter.png", roi, [cv2.IMWRITE_PNG_COMPRESSION, 0])
# filtered = sobel.filterImage(image, win)
# cv2.imwrite("Output/roi_after_filter.png", filtered, [cv2.IMWRITE_PNG_COMPRESSION, 0])


# Exercise 3 - Convolution on the Whole Image

# Exercise 4 - Different Convolution Kernels
